# Synthethic retail data generation

> *This notebook works well with the `Data Science 2.0` and `Data Science` kernels in SageMaker Studio*
>
> ⚠️ *You'll need to use a **2 vCPU + 16 GiB (`ml.m5.xlarge`)** or similar instance. The kernel will run out of RAM on the default 2 vCPU + 4 GiB (`ml.t3.medium`)*

This notebook generates a synthetic time series dataset to demonstrate demand forecasting in a retail context. The output (as detailed further in the [/README.md](../README.md)) includes:

- Daily sales data
- Static product metadata
- Dynamic (monthly) product procurement/manufacture costs
- Dynamic (daily) product sale prices
- Other dynamic demand-driving features, such as public holidays

## Import dependencies and configure parameters

We'll use some open-source libraries for time-series generation which aren't present in the standard SageMaker kernels, so you'll need to install those first if you haven't already:

In [ ]:
!pip install timeseries_generator workalendar

With the extra libraries installed, you're ready to import dependencies and set up configuration:

In [ ]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import os
import pickle  # nosec

# External Dependencies:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from timeseries_generator import (
    Generator,
    HolidayFactor,
    LinearTrend,
    RandomFeatureFactor,
    SinusoidalFactor,
    WeekdayFactor,
    WhiteNoise,
)
from timeseries_generator.external_factors import CountryGdpFactor

# Local Dependencies:
import util

# Set up data output folder (ignored by git):
os.makedirs('../dataset', exist_ok=True)

In [ ]:
# Increase default plotting size for readability: (width, height)
plt.rcParams['figure.figsize'] = (21, 6)

# Seed random number generator for reproducibility:
np.random.seed(seed=20060303)

In [ ]:
START_DATE = pd.Timestamp('2017-01-01')
END_DATE = pd.Timestamp('2019-12-31')

## Create factors

The dataset is built up by defining, and then later combining, multiple contributing **dynamic "factors"** based on **static "features"**: For example the *feature* "Country" could drive a *time-varying factor* "GDP", and potentially others as well.

Some of these features will affect demand directly, while others might drive item cost or pricing which drive demand indirectly.

In [ ]:
DEMAND_FEATURES = {}
DEMAND_FACTORS = {}
COST_FEATURES = {}
COST_FACTORS = {}

### Countries (GDP, holidays, and business growth)

Even single-country businesses are often affected by national holidays and macro-economic changes. In the example, we'll assume the business operates in a small group of countries with different conditions.

In [ ]:
countries_config = {
    'Brazil': {'growth': {'coef': 2, 'offset': -0.9}},
    'Ireland': {'growth': {'coef': 1.0, 'offset': 2.0}},
    'Japan': {'growth': {'coef': 0.4, 'offset': 1.6}},
    'Netherlands': {'growth': {'coef': 1.2, 'offset': -0.2}},
}

DEMAND_FEATURES['country'] = [k for k in countries_config.keys()]
COST_FEATURES['country'] = DEMAND_FEATURES['country']

In [ ]:
gdp_factor = CountryGdpFactor(country_list=DEMAND_FEATURES['country'])
DEMAND_FACTORS['gdp'] = gdp_factor
COST_FACTORS['gdp'] = gdp_factor
gdp_factor.plot(start_date=START_DATE, end_date=END_DATE)
plt.show()

In [ ]:
holiday_factor = HolidayFactor(country_list=DEMAND_FEATURES['country'])
holiday_factor.plot(start_date=START_DATE, end_date=END_DATE)
DEMAND_FACTORS['holidays'] = holiday_factor
COST_FACTORS['holidays'] = util.factors.transforms.scale_factor(
    holiday_factor,
    scale=0.5,
    base=1.0,
)
plt.show()

In [ ]:
ctry_growth_factor = LinearTrend(
    feature='country',
    feature_values={k: v['growth'] for k, v in countries_config.items()},
    col_name='ctry_growth',
)
DEMAND_FACTORS['ctry_growth'] = ctry_growth_factor
ctry_growth_factor.plot(start_date=START_DATE, end_date=END_DATE)
plt.show()

### In-country store locations

Most businesses will have more than one location in a particular country, so let's define a 'store' feature with a small set of possible values.

In our case we'll re-use store IDs between countries: So an individual 'location' identifier would be a combination of store and country:

In [ ]:
stores_config = {'Store1', 'Store2', 'Store3'}

DEMAND_FEATURES['store'] = sorted(s for s in stores_config)  # (Sort for reproducibility)

store_random_factor = util.factors.RandomCompositeFeatureFactor(
    feature_values={k: DEMAND_FEATURES[k] for k in ('country', 'store')},
    col_name='store_random',
    min_factor_value=0.1,
    max_factor_value=3,
)
DEMAND_FACTORS['store_random'] = store_random_factor
store_random_factor.plot(start_date=START_DATE, end_date=END_DATE)
plt.show()

### Weekday variation

For simplicity, we'll assume here that the same weekday factors can be applied across all countries where the business operates. Of course there may be countries and industries where this isn't true!

In [ ]:
weekday_factor = WeekdayFactor(
    col_name='weekend_boost_factor',
    factor_values={4: 1.15, 5: 1.3, 6: 1.3}  # Here we assign a factor of 1.15 to Friday, and 1.3 to Sat/Sun 
)
DEMAND_FACTORS['weekday'] = weekday_factor
weekday_factor.plot(start_date=START_DATE, end_date=END_DATE)
plt.show()

### Item popularity and seasonality

We'll define a set of actual product types here - each with:

- Some manually-configured seasonal variation
- A manually-configured base cost per item
- A randomly-selected long-term trend
- A randomly-selected short-term periodicity

In [ ]:
products_config = {
    'T-Shirt': util.config.ProductConfig(
        seasonality=util.config.SeasonalityConfig(
            amplitude=util.config.SeasonalityConfig.AMPLITUDE_DEFAULT / 0.5,
        ),
        base_cost=util.config.BaseCostConfig(initial=3.0, final_delta=0.4),
    ),
    'Hoodie': util.config.ProductConfig(
        seasonality=util.config.SeasonalityConfig(
            amplitude=util.config.SeasonalityConfig.AMPLITUDE_DEFAULT / 0.8,
            phase=util.config.SeasonalityConfig.PHASE_ANNUAL_WINTER,
        ),
        base_cost=util.config.BaseCostConfig(initial=8.0, final_delta=0.8),
    ),
    'Sneakers': util.config.ProductConfig(
        seasonality=util.config.SeasonalityConfig(
            wavelength=util.config.SeasonalityConfig.WAVELENGTH_2SEASONS,
        ),
        base_cost=util.config.BaseCostConfig(initial=10.0, final_delta=1.8),
    ),
    'Jeans': util.config.ProductConfig(
        seasonality=util.config.SeasonalityConfig(
            amplitude=util.config.SeasonalityConfig.AMPLITUDE_DEFAULT * .6,
            wavelength=util.config.SeasonalityConfig.WAVELENGTH_2SEASONS,
        ),
        base_cost=util.config.BaseCostConfig(initial=5.0, final_delta=0.7),
    ),
    'Scarf': util.config.ProductConfig(
        seasonality=util.config.SeasonalityConfig(
            amplitude=util.config.SeasonalityConfig.AMPLITUDE_DEFAULT * 1.8,
            phase=util.config.SeasonalityConfig.PHASE_ANNUAL_WINTER + 15,
        ),
        base_cost=util.config.BaseCostConfig(initial=3.0, final_delta=-.2),
    ),
    'Gloves': util.config.ProductConfig(
        seasonality=util.config.SeasonalityConfig(
            amplitude=util.config.SeasonalityConfig.AMPLITUDE_DEFAULT * 2,
            phase=util.config.SeasonalityConfig.PHASE_ANNUAL_WINTER + 30,
        ),
        base_cost=util.config.BaseCostConfig(initial=7.8, final_delta=1.2),
    ),
    'Sandals': util.config.ProductConfig(
        base_cost=util.config.BaseCostConfig(initial=3.2, final_delta=0.3),
    ),
    'Socks': util.config.ProductConfig(
        seasonality=util.config.SeasonalityConfig(
            phase=util.config.SeasonalityConfig.PHASE_ANNUAL_WINTER,
        ),
        base_cost=util.config.BaseCostConfig(initial=2.4, final_delta=0.3),
    ),
}

DEMAND_FEATURES['product'] = [p for p in products_config.keys()]
COST_FEATURES['product'] = DEMAND_FEATURES['product']

product_seasonality_factor = SinusoidalFactor(
    feature='product',
    col_name='product_seasonality',
    feature_values={
        name: cfg.seasonality.to_sinusoidal_factor_config()
        for name, cfg in products_config.items()
    },
)
DEMAND_FACTORS['product_seasonality'] = product_seasonality_factor
COST_FACTORS['product_seasonality'] = util.factors.transforms.scale_factor(
    product_seasonality_factor,
    scale=0.5,
    base=1.0,
)
product_seasonality_factor.plot(start_date=START_DATE, end_date=END_DATE)
plt.show()

In [ ]:
def gen_longterm_sinusoid():
    n_days_total = (END_DATE - START_DATE).days
    wavelength = np.random.uniform(n_days_total, n_days_total * 5)
    phase = np.random.uniform(-wavelength / 2, wavelength / 2)
    amplitude = np.random.uniform(
        util.config.SeasonalityConfig.AMPLITUDE_DEFAULT * 0.1,
        util.config.SeasonalityConfig.AMPLITUDE_DEFAULT * 2.,
    )
    mean = np.random.uniform(
        1. - amplitude / 2.,
        1. + amplitude / 2.,
    )
    return {'amplitude': amplitude, 'mean': mean, 'phase': phase, 'wavelength': wavelength}


product_longterm_factor = SinusoidalFactor(
    feature='product',
    col_name='product_longterm',
    feature_values={
        name: gen_longterm_sinusoid() for name in products_config.keys()
    },
)
DEMAND_FACTORS['product_longterm'] = product_longterm_factor
product_longterm_factor.plot(start_date=START_DATE, end_date=END_DATE)
plt.show()

In [ ]:
def gen_shortterm_sinusoid():
    wavelength = np.random.uniform(7, 90)
    phase = np.random.uniform(-wavelength / 2, wavelength / 2)
    amplitude = np.random.uniform(
        util.config.SeasonalityConfig.AMPLITUDE_DEFAULT * 0.1,
        util.config.SeasonalityConfig.AMPLITUDE_DEFAULT * 0.4,
    )
    mean = np.random.uniform(
        1. - amplitude / 2.,
        1. + amplitude / 2.,
    )
    return {'amplitude': amplitude, 'mean': mean, 'phase': phase, 'wavelength': wavelength}


product_shortterm_factor = SinusoidalFactor(
    feature='product',
    col_name='product_shortterm',
    feature_values={
        name: gen_shortterm_sinusoid() for name in products_config.keys()
    },
)
DEMAND_FACTORS['product_shortterm'] = product_shortterm_factor
product_shortterm_factor.plot(start_date=START_DATE, end_date=END_DATE)
plt.show()

In [ ]:
product_base_cost_factor = LinearTrend(
    feature='product',
    col_name='product_base_cost',
    feature_values={
        name: cfg.base_cost.to_linear_trend_config()
        for name, cfg in products_config.items()
    },
)
COST_FACTORS['product_base_cost'] = product_base_cost_factor
product_base_cost_factor.plot(start_date=START_DATE, end_date=END_DATE)
plt.show()

### Item size

Since our example items above are in a fashion context, let's add an 'item_size' attribute driving more variation:

> ⚠️ **Note:** `size` is a [reserved field name](https://docs.aws.amazon.com/forecast/latest/dg/reserved-field-names.html) not allowed in Amazon Forecast, hence we'll call this feature `item_size` to avoid potential problems later on.

In [ ]:
sizes_config = {
    'XL': {'offset': -.1},
    'L': {'offset': 0.5},
    'M': {'offset': 0.8},
    'S': {'offset': 0.1},
    'XS': {'offset': -0.3},
}
DEMAND_FEATURES['item_size'] = [k for k in sizes_config]


def size_factor_values_from_config(size_config):
    if not size_config:
        size_config = {}
    return {
        'coef': size_config.get('coef', 0.0),
        'offset': size_config.get('offset', 0.0),
    }


size_factor = LinearTrend(
    feature='item_size',
    feature_values={
        k: size_factor_values_from_config(v) for k, v in sizes_config.items()
    },
    col_name='size_trend',
)
DEMAND_FACTORS['size_trend'] = size_factor
size_factor.plot(start_date=START_DATE, end_date=END_DATE)
plt.show()

### Item color

...And similarly, a selection of colours in which each item might be available.

In [ ]:
colors_config = [
    'red', 'green', 'blue', 'black', 'yellow', 'pink', 'brown', 'white', 'gray', 'navy',
]

DEMAND_FEATURES['color'] = colors_config

color_random_factor = RandomFeatureFactor(
    feature='color',
    feature_values=DEMAND_FEATURES['color'],
    min_factor_value=0.1,
    max_factor_value=2.,
    col_name='color_factor',
)
DEMAND_FACTORS['color_factor'] = color_random_factor
color_random_factor.plot(start_date=START_DATE, end_date=END_DATE)
plt.show()

### Pricing actions / promotions

The sale price for each product will likely depend on its procurement/manufacture cost, plus some gross profit margin. Let's assume the margin is set by product type (independent of variations like size and colour), and doesn't directly drive demand because it's: A) static and B) based on the standard market price for the item.

In [ ]:
product_base_margin_factor = RandomFeatureFactor(
    feature='product',
    feature_values=DEMAND_FEATURES['product'],
    min_factor_value=1.1,
    max_factor_value=4.,
    col_name='product_base_margin',
)
product_base_margin_factor.plot(start_date=START_DATE, end_date=END_DATE)
plt.show()

Promotional discounts will be applied to products at various times through the year, and these certainly *should* drive changes in demand. In our model:

- The discount percentage will be inversely related to the effect on demand/sales
- The duration, amount, and distance between promotions per item will be randomly selected

In [ ]:
demand_promos_factor = util.factors.RandomPromotionsFactor(
    feature_values={'product': [name for name in products_config.keys()]},
    random_seed=123,
)
DEMAND_FACTORS['promos'] = demand_promos_factor

price_promos_factor = util.factors.transforms.scale_factor(
    util.factors.transforms.invert_factor(demand_promos_factor),
    scale=0.5,  # Price elasticity of demand (same across products in this simple case)
    base=1.0,  # Scaling happens around 1.0
)

demand_promos_factor.plot(start_date=START_DATE, end_date=END_DATE)
price_promos_factor.plot(start_date=START_DATE, end_date=END_DATE)
plt.show()

### Random noise

On top of these configured factors, some random noise will simulate natural, unpredictable variation in demand.

In [ ]:
WHITENOISE_STDDEV = 0.1

white_noise = WhiteNoise(stdev_factor=WHITENOISE_STDDEV)
DEMAND_FACTORS['noise'] = white_noise
white_noise.plot(start_date=START_DATE, end_date=END_DATE)
plt.show()

## Generate time-varying unit costs

With the features and factors defined, we'll first generate time-series of unit costs per item.

While the underlying factors affecting item cost may be daily in granularity, in practice the frequency of unit-cost changes a business observes may be limited: So we'll **aggregate unit costs by month** to illustrate this.

In [ ]:
%%time

util.config.generation_size_diagnostic(COST_FEATURES, COST_FACTORS, START_DATE, END_DATE)

g_cost = Generator(
    factors=COST_FACTORS.values(),
    features=COST_FEATURES,
    date_range=pd.date_range(start=START_DATE, end=END_DATE),
    base_value=1.0,
)

print('\nGenerating item unit costs...')
df_cost = g_cost.generate()

print('Enforcing minimum unit cost value...')
df_cost[df_cost['value'] < 0.1] = 0.1

print('Grouping by month...')
df_cost = df_cost.groupby(
    [*COST_FEATURES.keys(), pd.PeriodIndex(df_cost['date'], freq='M')]
).agg({'value': 'mean'})
df_cost = df_cost.rename(columns={'value': 'unit_cost'})
df_cost['unit_cost'] = df_cost['unit_cost'].round(decimals=2)

display(df_cost)
df_cost.describe()

In [ ]:
print('Saving generated cost data...')

# Need to reset index for parquet or Pandas will not load the date index column correctly:
df_cost.reset_index().to_parquet('../dataset/unit_costs.parquet')

print('Done')

We'll assume unit cost is a factor driving sale price (below), but can't just directly reference the `COST_FACTORS` because we want to base price on the aggregated monthly figures (that our business would see from its suppliers), not the underlying variables (that our business would not directly see).

Therefore we'll define this aggregated cost dataframe as a new kind of factor for the timeseries generator:

In [ ]:
item_cost_factor = util.factors.ExternalDateAggregatedFactor(
    df_cost,
    col_name="unit_cost",
    features=COST_FEATURES,
)

item_cost_factor.plot(start_date=START_DATE, end_date=END_DATE)

## Generate item prices from cost and other factors

We'll model the sale price of each item as being derived from the underlying unit cost, plus a margin, minus any promotional discounts.

Note that if using product prices as an input to demand forecasting, you'll want to project out a scenario for future prices as well: Similarly to how we'll extend out holiday and weekend reference data beyond the base `END_DATE` in a later section. However, projecting out pricing scenarios is a bit more of an opinionated business decision than looking up public reference data: So we'll build this source data to stop at `END_DATE` and rely on forecasters to extend from there themselves.

In [ ]:
%%time
PRICE_FEATURES = {**COST_FEATURES}
PRICE_FACTORS = {
    'unit_cost': item_cost_factor,
    'base_margin': product_base_margin_factor,
    'promos': price_promos_factor,
}

util.config.generation_size_diagnostic(PRICE_FEATURES, PRICE_FACTORS, START_DATE, END_DATE)

g_price = Generator(
    factors=PRICE_FACTORS.values(),
    features=PRICE_FEATURES,
    date_range=pd.date_range(start=START_DATE, end=END_DATE),
    base_value=1.0,
)

print('\nGenerating item price data...')
df_price = g_price.generate()
df_price = df_price.rename(columns={'value': 'unit_price', 'random_promos_factor': 'promo'})

# Round output fields to avoid weirdly precise outputs:
df_price['unit_price'] = df_price['unit_price'].round(decimals=2)
df_price['promo'] = df_price['promo'].round(decimals=3)

display(df_price)
df_price.describe()

Since items are *usually* sold at their regular price, we can take a quick diagnostic of how many records are impacted by promotions and what the typical discount percentages are:

In [ ]:
df_price[df_price['promo'] != 1.0]['promo'].describe()

When happy, this can be saved as our standard item price dataset:

In [ ]:
print('Saving generated price data...')
price_export_cols = ["date", *PRICE_FEATURES.keys(), 'promo', 'unit_price']
df_price[price_export_cols].to_parquet('../dataset/prices_promos.parquet', index=False)
print('Done')

We won't need this dataframe again in the notebook: Since price is fully under the business' control, we can just re-reference the underlying `PRICE_FACTORS` for sales generation. Hence we can delete the variable to free some memory:

In [ ]:
del df_price

## Generate underlying demand

With cost and pricing factors calculated, we're ready to simulate the *underlying demand for sales* for each item by day. (This is not quite the same as the final synthetic sales data, as we'll show in later sections)

Setting the `base_value` controls the overall **sparsity** of the dataset, which is useful for simulating common retail patterns where e.g. there may be a long tail of SKUs with low sales.

In [ ]:
base_value = .8

With all the factors defined and a base value configured, we're ready to generate the time-series:

In [ ]:
%%time
util.config.generation_size_diagnostic(DEMAND_FEATURES, DEMAND_FACTORS, START_DATE, END_DATE)

g = Generator(
    factors=DEMAND_FACTORS.values(),
    features=DEMAND_FEATURES,
    date_range=pd.date_range(start=START_DATE, end=END_DATE),
    base_value=base_value,
)

print('\nGenerating demand data...')
df = g.generate()

print('Enforcing non-negative integer output...')
df['value'] = df['value'].astype(int)
df[df['value'] < 0]['value'] = 0  # (This is not necessary unless any factors might have gone -ve)

df

We'll save this raw file and the input configurations directly:

In [ ]:
print('Saving configuration to file...')
with open('../dataset/generator_config.pkl', 'wb') as fpkl:
    pickle.dump(  # nosec
        {
            'cost_factors': COST_FACTORS,
            'cost_features': COST_FEATURES,
            'demand_factors': DEMAND_FACTORS,
            'demand_features': DEMAND_FEATURES,
            'price_factors': PRICE_FACTORS,
            'price_features': PRICE_FEATURES,
            'start': START_DATE,
            'end': END_DATE,
            'base_value': base_value,
        },
        fpkl,
    )

print('Saving generated data...')
df.to_parquet('../dataset/demand_raw.parquet', index=False)

### (Load a previous generation)

At this point we'll re-load the raw generation result from above:

1. To check the integrity, and
2. To provide a useful place to resume, in case later experimentation causes your kernel to run out of memory and restart!

In [ ]:
print('Loading generated dataframe')
df = pd.read_parquet('../dataset/demand_raw.parquet')

with open('../dataset/generator_config.pkl', 'rb') as fpkl:
    print('Loading configuration from file')
    obj = pickle.load(fpkl)  # nosec
    COST_FACTORS = obj['cost_factors']
    COST_FEATURES = obj['cost_features']
    DEMAND_FACTORS = obj['demand_factors']
    DEMAND_FEATURES = obj['demand_features']
    PRICE_FACTORS = obj['price_factors']
    PRICE_FEATURES = obj['price_features']
    START_DATE = obj['start']
    END_DATE = obj['end']
    base_value = obj['base_value']

print('Ready!')

In [ ]:
print('Statistics of daily demand column:')
print(df['value'].describe(), end='\n\n')

n_zero_demands = (df['value'] == 0).sum()
print(
    'Zero sales on: {} records ({:.1%} of total)'.format(n_zero_demands, n_zero_demands / len(df))
)

df

## From demand to sales

The raw generated demand table above needs some further transformation to create something like a sales database.

### Basic data shape

First, from a functional perspective, the working columns where underlying factors were calculated of course shouldn't be exposed to forecasting. We also wouldn't see any record generated at all in sales systems on days where a particular item didn't sell *any* units:

In [ ]:
df.drop(
    columns=['base_amount', 'total_factor'] + [f.col_name for f in DEMAND_FACTORS.values()],
    inplace=True,
)
df.rename(columns={'value': 'demand'}, inplace=True)
df = df.loc[df['demand'] != 0].copy()
df

### Dropping extreme low-volume items

Second, from a business perspective, since the generation was random and sparsified there may be (location, product, variant) combinations with extremely low demand throughout the entire period.

In practice, a most businesses simply wouldn't carry such extremely low-demand products over extended periods with regular re-ordering. A more likely scenario would be an initial trial order with no follow-up once demand was found to be too weak.

We won't try to model new product introduction complexities in this sample, so will simply drop these *extreme* sparse cases.

You could consider filtering either on the frequency of days having any sales (count), or the total number of sales (sum), or a combination of the two. We'll use the frequency of days, and drop any item/location combinations that were very rarely sold across the entire multi-year period:

In [ ]:
demand_dims = [c for c in df.columns if c not in ('date', 'demand')]
demand_record_counts = df.groupby(demand_dims)['demand'].count().sort_values()
demand_record_counts

In [ ]:
extreme_rare_sales = pd.Series(
    True,
    index=demand_record_counts[demand_record_counts < 10].index,
    name='drop_rare',
)
print(
    'Dropping %s extremely rarely-selling item/location combinations (out of %s total)'
    % (len(extreme_rare_sales), len(demand_record_counts))
)
extreme_rare_sales

In [ ]:
df = pd.merge(df, extreme_rare_sales.to_frame().reset_index(), how='left')
df = df.loc[df['drop_rare'].isna()]
df.drop(columns=['drop_rare'], inplace=True)
df

### (Theory only) Modelling historical stock-outs

For a really realistic treatment of inventory-related demand forecasting benefits, we should recognize that the business doesn't keep infinite stock on hand for every item, and therefore has probably *run out of stock* at various periods in the history.

This would require simulating a baseline stock re-ordering process (such as a moving average demand forecast plus some margin and/or minimum "safety stock" level) through the demand history and demand for sales that couldn't be fulfilled.

The resulting historical "inventory available" (0/1) status would be an important input feature for a demand forecasting model, as the model would need to learn the difference between periods of no sales due to low demand, versus periods of no sales where latent demand was high but stock was not available.

We haven't yet implemented this stock-out modelling, but acknowledge it could be useful in future for improving realism.

In [ ]:
df.rename(columns={'demand': 'sales'}, inplace=True)

## Extract and normalize datasets

Having mapped from raw generated "demand" to a more realistic dataset of "sales", we're almost ready to save our result. All that remains is some field normalization:

Usually, businesses will have some kind of item ID (for example the Stock Keeping Unit or SKU) which uniquely identifies a particular product variant. Likewise a store/location ID may or may not be unique across geographies.

With Amazon Forecast in particular, there's a decision to be made between:

- Splitting "items" as unique location-SKU combinations, to allow setting different metadata for one product across multiple locations, or
- Treating SKU/product as the "item" identifier and location as a "dimension" of the forecast - so that one product shares metadata across all sale locations

Either can be viable approaches, mainly depending whether there's a need to configure separate product metadata for different locations or not.

In this sample we'll assume the only static product metadata used in forecasting is consistent across all locations - and use a single separate dimension to represent store location. In both cases the ID field will just be a concatenation of the underlying features - so you'll be able to recover the detail easily later if needed.

In [ ]:
location_features = ['country', 'store']
location_aggregations = {'location': {'features': location_features, 'sep': '_'}}
sku_features = ['product', 'color', 'item_size']
sku_aggregations = {'sku': {'features': sku_features, 'sep': '_'}}

First, create our "item master" product metadata table (from the sales data, to omit any products that were too sparse to feed through):

In [ ]:
%%time
# De-duplicate all features (size faster than other group iterating methods):
metadata_df = df.groupby([c for c in demand_dims if c not in location_features]).size()

# Convert from a MultiIndex to df of values
metadata_df = metadata_df.index.to_frame(index=False)

# Aggregate the features together into a 'SKU' identifier:
util.analytics.agg_df_features(metadata_df, sku_aggregations, drop_aggregated=False)

metadata_df

A double-check just to make sure this identifier is unique:

In [ ]:
# Check SKU uniquely identifies records:
n_skus = len(metadata_df['sku'].unique())
if n_skus != len(metadata_df):
    raise ValueError('sku does not uniquely identify records in the product metadata table!')

...And if all's well, this static product metadata is ready to save and use:

In [ ]:
metadata_df.to_csv('../dataset/metadata.csv', index=False)
metadata_df

Then, likewise apply aggregations to our sales data so that it's keyed only on the 'sku' product identifier and the combined location identifier:

In [ ]:
util.analytics.agg_df_features(df, sku_aggregations, drop_aggregated=True)
util.analytics.agg_df_features(df, location_aggregations, drop_aggregated=True)
df

At this point it may be worth checking the **overall sparsity** of the generated sales data, as this will affect the difficulty of the forecasting task. We can produce logarithmic plots to describe how many location-SKU combinations have at least N days with non-zero sales data through the history, and how many have at least X total units sold through the history:

In [ ]:
plt.figure()
axfreq = util.analytics.log_log_plot(
    df.groupby(['sku', 'location'])['sales'].count().value_counts(),
    title='Coverage over historical time period',
    xlabel='Number of SKU-locations',
    ylabel='Minimum days with sales',
    # xnorm=True,  # Can optionally normalize from absolute count to 0-1 proportion of dataset
)
axfreq.get_figure().savefig('../dataset/item-coverage.png', bbox_inches='tight')

plt.figure()
axvol = util.analytics.log_log_plot(
    df.groupby(['sku', 'location'])['sales'].sum().value_counts(),
    title='Total sales volume',
    xlabel='Number of SKU-locations',
    ylabel='Minimum total units sold',
    # xnorm=True,  # Can optionally normalize from absolute count to 0-1 proportion of dataset
)
axfreq.get_figure().savefig('../dataset/item-volume.png', bbox_inches='tight')

Assuming the data seems healthy, we at last we have our simulated sales system source data:

In [ ]:
df.to_parquet('../dataset/sales.parquet', index=False)
df

In [ ]:
del df

## Extract weekend and holiday dataset

In addition to **static** item metadata, we'll later want to use some **dynamic** features to help build better forecasts.

Public holidays were accounted for in the original timeseries generation, but the computed factor value is smoothed over multiple days by the library. It would be a bit of a cheat to use that representation directly in forecasting - so instead here we'll use the same underlying source to extract a list of holidays per country, and produce a more basic/custom dataset.

In [ ]:
from workalendar.america import Brazil
from workalendar.asia import Japan
from workalendar.europe import Ireland, Netherlands

country_calendars = {
    'Brazil': Brazil(),
    'Ireland': Ireland(),
    'Japan': Japan(),
    'Netherlands': Netherlands(),
}

print('Countries in dataset:')
print(DEMAND_FEATURES['country'])

for country in DEMAND_FEATURES['country']:
    if country not in country_calendars:
        raise ValueError(f'Need to manually configure workalendar country "{country}" import')

Note that, since our actual historical sales data goes to `end_date`, we'll want holiday data to extend beyond that and cover the forecast horizon - to make it a useful input for the forecasting model.

The underlying public holiday list can only be fetched per year:

In [ ]:
def build_holiday_list(countries, start_date, end_date):
    """Doesn't actually filter by date - just year"""
    dfs = []
    for country in countries:
        for year in range(start_date.year, end_date.year + 1):
            cal = country_calendars[country].holidays(year)
            dfs.append(
                pd.DataFrame({
                    'country': country,
                    'date': pd.Series([hol[0] for hol in cal]).unique(),
                })
            )
    return pd.concat(dfs, axis=0)


holidays_df = build_holiday_list(
    DEMAND_FEATURES['country'],
    START_DATE,
    END_DATE + pd.DateOffset(years=1),  # Include one year after the sales data ends
)
holidays_df

To build the daily reference table, we'll first set up the days of the weekends (which are the same across countries in this example):

In [ ]:
weekend_flag_df = WeekdayFactor(
    col_name='weekend_flag',
    factor_values={0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1},
).generate(
    start_date=START_DATE,
    end_date=END_DATE + pd.DateOffset(years=1, days=1),  # (Skips final Dec 31st without the offset)
)
weekend_flag_df

...Then, add the holiday days by country as overrides in the table:

In [ ]:
holiday_overrides = holidays_df.set_index(['country', 'date'])
holiday_overrides['holiday_flag'] = 1

# Replicate the weekend_flag_df across countries and set up its index:
weekend_hol_df = weekend_flag_df.merge(
    pd.Series(DEMAND_FEATURES['country'], name='country'),
    how='cross',
).set_index(['country', 'date'])

# Join to holiday_overrides with the MultiIndex on [country, date]:
weekend_hol_df = weekend_hol_df.join(holiday_overrides, how='left')
weekend_hol_df['holiday_flag'] = weekend_hol_df['holiday_flag'].fillna(0).astype(int)

# Create a composite field that's 2 on holidays, 1 on weekends, 0 otherwise:
weekend_hol_df['weekend_hol_flag'] = np.where(
    weekend_hol_df['holiday_flag'],
    2,
    weekend_hol_df['weekend_flag'],
)
weekend_hol_df

The combined feature is now ready to export:

In [ ]:
weekend_hol_df.drop(columns=['weekend_flag', 'holiday_flag']).to_csv(
    '../dataset/weekend_holiday_flag.csv'
)

## All done!

The source data files are now saved in the [../dataset](../dataset) folder and ready to use.

> ⚠️ **Note:** We've done our best to seed random number generators in this process for **reproducibility** assuming you restart the kernel before re-running the cells, and run the full notebook through start-to-end. However, it's better to be safe than sorry: Make sure to keep track of your dataset versions if experimenting with this code, and don't mix and match output files from different runs!